In [759]:
### Analyze MOFA MODEL Results
### Associate the generated factors with sample meta-data covariates and plot the top features per factor

#############################################
# Prerequisites - Load Libraries

In [760]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/mofa_analysis//lib/R/library"


In [761]:
source('MS2_Plot_Config.r')

In [762]:
source('MS1_Functions.r')

###############################################
# Preqrequisites Configurations & Parameters

In [763]:
### Load the parameters that are set via the configuration files

In [764]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [765]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/example_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/example_results/


In [766]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [767]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_data/"

In [768]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [769]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_results/"

In [770]:
## Downstream Analysis Configurations (mainly specifying the covariates that should be analyzed in relation to the MOFA factors)

In [771]:
factor_configs = read.csv( 'configurations/04_Factor_Analysis_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/04_Factor_Analysis_Configs.csv'”


In [772]:
head(factor_configs,2)

,configuration_name,mofa_result_name,relevant_factors,numeric_covariates,categorical_covariates,top_variable_thres.
,<chr>,<chr>,<chr>,<lgl>,<chr>,<dbl>
1,MI_v1,MI_v1_MOFA,"Factor1,Factor2,Factor3,Factor4,Factor5,Factor6",NA,"disease,sex,Primary.Genetic.Diagnosis",0.005


In [773]:
## Get Type color codes from previous script

In [774]:
type_color_codes = read.csv( 'configurations/03_Type_Color_Codes.csv', sep = ',')

In [775]:
head(type_color_codes,2)

,X,color_code
,<int>,<chr>
1,1,#FF7F50
2,2,#D95F02


In [776]:
type_color_codes$color_code

[1] "#FF7F50" "#D95F02" "#377EB8" "#E6AB02" "#31A354" "#7570B3" "#E7298A"
[8] "#66A61E" "#A6761D"

# Load Data 

## MOFA Input

In [777]:
#### Load the data based on which the MOFA model was generated (from step 02)

In [778]:
data_list = list()

In [779]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    data_name = factor_configs$configuration_name[i]
    
    # Load data
    path = paste0(result_path, '/02_results/02_Combined_Data_',data_name,'_INTEGRATED.csv')
    data_long = read.csv(path)
    data_long$X = NULL
    print(file.info(path)$mtime)
    print(path)
    
    # save to list
    data_list[[i]] = data_long
    }

[1] "2024-05-10 22:27:00 CEST"
[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_results//02_results/02_Combined_Data_MI_v1_INTEGRATED.csv"


In [780]:
## Example of loaded data

In [781]:
head(data_long,2)

,sample_id,variable,value,type,gene
,<chr>,<chr>,<dbl>,<chr>,<chr>
1,D1,adipocyte__ABI1,-1.1243382,adipocyte,ABI1
2,D2,adipocyte__ABI1,-0.8572543,adipocyte,ABI1


## MOFA Model

### Load estimated model

In [782]:
### Load the MOFA model(s) that should be analyzed

In [783]:
model_list = list()

In [784]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    mofa_name = factor_configs$mofa_result_name[i]
 
    # Load model
    model_name =  paste0("03_MOFA_MODEL_",mofa_name, '.hdf5')
    outfile = file.path( paste0(result_path, '/03_results/',  model_name) )
    model = load_model(outfile, verbose = TRUE)
    
    print(file.info(outfile)$mtime)
    print(outfile)
    
    # save to list
    model_list[[i]] = model
    }
    

Loading data...

Loading expectations for 2 nodes...

Loading model options...

Loading training options and statistics...

Assigning names to the different dimensions...

Re-ordering factors by their variance explained...

Doing quality control...

Checking views names...

Checking groups names...

Checking samples names...

Checking features names...

Checking dimensions...

Checking there are no features with complete missing values...

Checking sample covariates...

Checking expectations...

Checking for intercept factors...

Warning message in .quality_control(object, verbose = verbose):
“Factor(s) 2, 8 are strongly correlated with the total number of expressed features for at least one of your omics. Such factors appear when there are differences in the total 'levels' between your samples, *sometimes* because of poor normalisation in the preprocessing steps.
”
Checking for highly correlated factors...



[1] "2024-05-10 22:28:01 CEST"
[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_results//03_results/03_MOFA_MODEL_MI_v1_MOFA.hdf5"


### Factor Data

In [785]:
### Load the factor data to the corresponding MOFA model

In [786]:
factor_data_list = list()

In [787]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    mofa_name = factor_configs$mofa_result_name[i]
    
    # load data
    path = paste0(result_path, '/03_results/', '03_Factor_Data_', mofa_name, '.csv')
    factors = read.csv(path)
    print(file.info(path)$mtime)
    print(path)
    
    # Save to list
    factor_data_list[[i]] = factors
    }
    

[1] "2024-05-10 22:28:01 CEST"
[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_results//03_results/03_Factor_Data_MI_v1_MOFA.csv"


In [788]:
head(factors,2)

,Factor1,Factor2,Factor3,Factor4,Factor5,Factor6,Factor7,Factor8,Factor9,Factor10,sample_id
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1.616970,1.922963,-0.04046481,0.6957385,-0.4816992,-0.6951516,0.6601078,-0.1757625,0.04568479,-0.2038762,D1
2,1.749186,0.227296,0.60221439,-0.1048832,-0.7551822,-1.5846769,-0.8970750,-0.1889143,0.22074658,-0.8048890,D2


### Weight Data

In [789]:
# Load the feature factor weights to the corresponding MOFA model

In [790]:
weight_data_list = list()

In [791]:
for(i in 1:nrow(factor_configs)){
    # get data name from configuration
    mofa_name = factor_configs$mofa_result_name[i]
    
    # laod data
    path = paste0(result_path, '/03_results/', '03_Weight_Data_',  mofa_name, '.csv')
    weight_data = read.csv(path)
    
    print(file.info(path)$mtime)
    print(path)
    
    # save to list
    weight_data_list[[i]] = weight_data
    }
    
    

[1] "2024-05-10 22:28:01 CEST"
[1] "/lustre/groups/epigenereg01/workspace/projects/jove/example_results//03_results/03_Weight_Data_MI_v1_MOFA.csv"


## Sample Meta Data

In [792]:
## Load the sample meta-data file that contains covariates that should be associated to factor data

In [793]:
sample_data = read.csv(paste0(data_path, 'Prepared_Sample_Meta_Data', '.csv'))
sample_data$X = NULL

In [794]:
head(sample_data,2)

,donor_id,disease,sex,development_stage,self_reported_ethnicity,Primary.Genetic.Diagnosis,sample_id
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,DP2,dilated cardiomyopathy,female,fifth decade human stage,European,PLN,DP2
2,H3,normal,male,sixth decade human stage,Asian,control,H3


# Downstream Analysis of generated model

# Extract and prepare data for plots

## Merge factors and sample data

In [795]:
## Combine the factor and sample data table

In [796]:
factor_data_processed = list()

In [797]:
for( i in 1:length(factor_data_list)){
    
    ## Transform data to long format
    factors_long = melt(factor_data_list[[i]], id.vars = 'sample_id')
    
    ## Add sample data info
    merged_data_long = merge(factors_long, sample_data, by.x = 'sample_id', by.y = 'sample_id')
    
     ### Filter on relevant factors
     relevant_factors = unlist(str_split(factor_configs$relevant_factors[i], ','))  # get relevant factor from configuration data
     merged_data_long = merged_data_long[merged_data_long$variable %in% relevant_factors,]
    
     factor_data_processed[[i]] = merged_data_long
    }

In [798]:
head(factor_data_processed[[i]],2)

,sample_id,variable,value,donor_id,disease,sex,development_stage,self_reported_ethnicity,Primary.Genetic.Diagnosis
,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,D1,Factor1,1.616970,D1,normal,female,sixth decade human stage,European,control
2,D1,Factor2,1.922963,D1,normal,female,sixth decade human stage,European,control


## Extract explained variance for plotting

In [799]:
## Get the explained variance information from the MOFA model (these values will be plotted later)

In [800]:
explained_variance = lapply(model_list, function(x){
    
    # extract variance per factor from model
    data = x@cache$variance_explained$r2_per_factor[[1]]
    
    # extract total variance from model
    total_variance = data.frame( view = rownames(x@cache[["variance_explained"]]$r2_total$group1,2),
                             total_variance = x@cache[["variance_explained"]]$r2_total$group1)
    
    # extract total variance per factor
    total_variance_factor = data.frame(factor = names(rowMeans(x@cache$variance_explained$r2_per_factor[[1]])),
                                   mean_variance = rowMeans(x@cache$variance_explained$r2_per_factor[[1]]))
    
    data = melt(data)
    # merge different variance values
    data = merge(data, total_variance, by.x = 'Var2', by.y = 'view')
    
    })

In [801]:
head(explained_variance[[1]],2)

,Var2,Var1,value,total_variance
,<fct>,<fct>,<dbl>,<dbl>
1,adipocyte,Factor1,6.647976,32.79514
2,adipocyte,Factor2,2.465791,32.79514


## Prepare weight data

In [802]:
### Adjust the format of the feature factor weight data to long

In [803]:
feature_weights_list = lapply(weight_data_list, function(x){
    feature_weights_long = melt(x, id.vars = c('variable_name', 'type'))
    
    # adjust formatting of columns
    feature_weights_long$view = feature_weights_long$type
    feature_weights_long$gene = str_replace(feature_weights_long$variable_name, '.*__', '')
    
    return(feature_weights_long)
    })
    

In [804]:
head(feature_weights_list[[1]],2)

,variable_name,type,variable,value,view,gene
,<chr>,<chr>,<fct>,<dbl>,<chr>,<chr>
1,adipocyte__ABI1,adipocyte,Factor1,-0.16729734,adipocyte,ABI1
2,adipocyte__ABI2,adipocyte,Factor1,0.04293374,adipocyte,ABI2


## Get top features per factor and amounts for diff thresholds

In [805]:
## Get the x% of top features per factor based on the specified threshold (x) in the configuration file

In [806]:
geneset_oi_amounts_list = list()

In [807]:
for(i in 1:length(feature_weights_list)){
    
    feature_weights_long = feature_weights_list[[i]]
    
    # get threshold for top variables from configuration
    top_variable_perc = factor_configs$top_variable_thres[i]
    
    
    ## Define amount of top genes per fraction 
    geneset_oi_pos_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange( desc(value),  .by_group = TRUE)  %>% top_frac(  as.numeric(top_variable_perc), value)
    geneset_oi_pos_per_factor_analyze$direction = 'positive'
    
    geneset_oi_neg_per_factor_analyze = feature_weights_long %>% group_by(variable) %>% dplyr::arrange(desc(value),  .by_group = TRUE)  %>% top_frac( - as.numeric(top_variable_perc), value)
    geneset_oi_neg_per_factor_analyze$direction = 'negative'
    
    geneset_oi_analyze = rbind(geneset_oi_pos_per_factor_analyze, geneset_oi_neg_per_factor_analyze)
    geneset_oi_analyze$fraction =  as.numeric(top_variable_perc)

    
    ## Calculate the amount of top features per type
    dimensions = unique(feature_weights_long[,c('view', 'variable')])
    
    amount_geneset_oi_type = geneset_oi_analyze %>% group_by(type, view, variable) %>% dplyr::count()
    amount_geneset_oi_type = merge(dimensions, amount_geneset_oi_type, all.x = TRUE) # to avoid missing dimensions
    amount_geneset_oi_type$fraction = as.numeric(top_variable_perc)
    
    geneset_oi_amounts = amount_geneset_oi_type
    
    ## Calculate the total amount of features per type
    features_per_type = feature_weights_long %>% group_by(type, view, variable) %>% dplyr::count()
    colnames(features_per_type) = c('type', 'view', 'variable', 'amount_features')
    
    ## Merge and calculate percentage
    geneset_oi_amounts = merge(  geneset_oi_amounts,features_per_type, all.x = TRUE)
    geneset_oi_amounts$percentage = geneset_oi_amounts$n / geneset_oi_amounts$amount_features
    
    ## Adjust NA's
    geneset_oi_amounts[is.na(geneset_oi_amounts)] = 0 # NA when there are no features for this dimension among top %
    
    ## save for the threshold/ config
    geneset_oi_amounts_list[[i]] = geneset_oi_amounts
    }
    
    
    
    

In [808]:
head(    geneset_oi_amounts_list[[i]],2)

,view,variable,type,n,fraction,amount_features,percentage
,<chr>,<fct>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,adipocyte,Factor1,0,0,0.005,0,0
2,adipocyte,Factor2,0,0,0.005,0,0


# Plots

In [809]:
### Generate the plots to evaluate the factor values in relation to different sample-meta data covariates

## Investigate relationship of factors with numeric values

In [810]:
### Calculate correlatins with numeric features

In [811]:
head(factor_data_processed[[1]],2)

,sample_id,variable,value,donor_id,disease,sex,development_stage,self_reported_ethnicity,Primary.Genetic.Diagnosis
,<chr>,<fct>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,D1,Factor1,1.616970,D1,normal,female,sixth decade human stage,European,control
2,D1,Factor2,1.922963,D1,normal,female,sixth decade human stage,European,control


In [812]:
figure_name = "FIG04_Factor_Association_with_numeric_features_"  # Define the name of the plot

In [813]:
# Set the sizes of the plot
width_par = 8.07
height_par = 3.5

In [814]:
for(j in 1:length(factor_data_processed)){
    ## get the variables
    numeric_variables = unlist(str_split(factor_configs$numeric_covariates[j], ','))
    
    
    ## calculate correlations
    cor_plot = list()
    
    ## get the data
    merged_data_long = factor_data_processed[[j]]
    
    ### calculate correlations
    for(i in numeric_variables){
        cor_plot[[i]] = ggplot(merged_data_long, aes(x = value, y = get(i))) + facet_wrap(.~ variable, scale = 'free') +
        geom_point(size = 0.2) + plot_config + geom_smooth(method='lm', col = 'blue3', se = FALSE) + stat_cor(method = 'pearson') + ylab(i)


        }
    
    
    ### Plot the scatterplots
    
    # get the name for saving
    mofa_name = factor_configs$mofa_result_name[j]
    
    pdf(paste0('figures/04_figures/', figure_name,mofa_name, '.pdf'), width =width_par, height =height_par, onefile = TRUE)
    for (i in names(cor_plot)) {
      print(cor_plot[[i]])  
    }
    dev.off()
    }
    
    

NULL


## Investigate relationship with categorical values

In [815]:
### Generate boxplots to evaluate the factor value difference for categorical sample meta data variables

In [816]:
# Specific Text Descriptions:
xlabel = xlab('') 
ylabel = ylab('Factor Value')

In [817]:
# Specify Figure Name
figure_name = paste0("FIG04_Factor_Association_with_categorical_features_")

In [818]:
# Specify sizes of the plot
width_par = 8.07
height_par = 3

In [819]:
for(j in 1:length(factor_data_processed)){
    ## get the variables
    categorical_variables = unlist(str_split(factor_configs$categorical_covariates[j], ','))
    
    
    ## calculate correlations
    cor_plot = list()
    
    ## get the data
    merged_data_long = factor_data_processed[[j]]
    
    
    ## Plot as PDF
    mofa_name = factor_configs$mofa_result_name[j]
    
    pdf(paste0('figures/04_figures/', figure_name, mofa_name, '.pdf'), width =width_par, height =height_par)
    for(i in categorical_variables){
        variable = i
        merged_data_long$condition = merged_data_long[,variable]
        vis_data = merged_data_long
        g = ggplot(vis_data[!is.na(vis_data$condition),], aes(x=condition, y=value, col = condition)) + facet_grid(.~variable) +
            plot_config +
            xlabel + 
            ylabel + #colors_list[[i]]   +
            ggtitle('Pattern of factor values') + 
            theme(legend.position = "bottom", axis.text.x = element_blank())+
            geom_boxplot(outlier.size = 0.05)   + geom_point(size = 0.5) + if(i %in% names(colors_list)){colors_list[[i]]} else{colors_list[['default']]}

        print(g)

        }

    dev.off()
    
    }

## Feature Overview per Factor

In [820]:
## Generate the overview of the amount of top x% of features per factor and view (in relation to total amount of features per view)

In [821]:
## Plot the heatmap showing the explained variance of the factor per view

In [822]:
variance_plots = list()

In [823]:
for(j in 1:nrow(factor_configs)){

    ## get the relevant factor and top variable fraction
    factor_var = unlist(str_split(factor_configs$relevant_factors[j], ','))
    
    
    ## Explained Variance Heatmap Plot (for each factor)
    explained_variance_heatmap = list()
    for(i in factor_var ){
        data = explained_variance[[j]]   # get prepared variance data
        data$Var2 = as.character(data$Var2)
        data$Var2 = factor(data$Var2, levels = sort(unique(data$Var2)))  # recode to ensure right ordering
        
        data_plot = data[data$Var1 == i,]
        data_plot$Var1 = 'Explained'

        explained_variance_heatmap[[i]] = ggplot() + scale_fill_gradient(low="white", high="black") + 
        ylabel + xlabel + plot_config_heatmap +  theme(axis.text.y = element_text(hjust = 0, vjust = 0.5)) +
        geom_tile(data = data_plot, mapping = aes(Var1,  Var2, fill= value)
                 )  +
        ggtitle(i)
        }
    
    variance_plots[[j]] =  explained_variance_heatmap
    }

In [824]:
#variance_plots[[1]]

In [825]:
## Generate the barplots with showing the amount of top x% of features

In [826]:
barplot_top_features_percentages = list()
barplot_top_features_absolute = list()

In [827]:
for(j in 1:nrow(factor_configs)){
    
    # get configurations
    top_var_fraction = factor_configs$top_variable_thres[j]
    factor_var = unlist(str_split(factor_configs$relevant_factors[j], ','))
    
    ## Barplots with top features per factor
    
    ## 1: Percentages (amount of top x% of features in relation to total amount)
    xlabel = xlab('View') 
    ylabel = ylab('Percentage of total features')
    
    percentage_plot_1_perc = list()
    
    for(i in factor_var ){

            geneset_oi_amounts = geneset_oi_amounts_list[[j]]
            geneset_oi_amounts$view  = as.character(geneset_oi_amounts$view)
            geneset_oi_amounts$view = factor(geneset_oi_amounts$view, levels = sort(unique(geneset_oi_amounts$view))) # recode to ensure right ordering

            percentage_plot_1_perc[[i]] = ggplot(data = geneset_oi_amounts[(geneset_oi_amounts$variable == i),], aes(x = view, y = percentage*100, fill = view)) +
            xlabel + 
            ylabel + 
            plot_config + 
            geom_bar(stat="identity") + coord_flip() + theme(legend.position = 'none') +
            ggtitle(paste0('Top ', 2*as.numeric( top_var_fraction) *100, '% of features')) +
            geom_hline(yintercept = 2*as.numeric( top_var_fraction)*100, 
                    color = "black", size=1) + scale_fill_manual(values = type_color_codes$color_code)
        
    }
    barplot_top_features_percentages[[j]] =  percentage_plot_1_perc 
    
    
    
    ## 2: Absolute Values (absolute amount of top x% of features of the view)
    
    xlabel = xlab('View') 
    ylabel = ylab('Amount features')
    
    absolute_plot_1_perc = list()
    
    # one selected threshold + absolute amount
    for(i in factor_var ){
            geneset_oi_amounts = geneset_oi_amounts_list[[j]]
            geneset_oi_amounts$view = as.character(geneset_oi_amounts$view)
            geneset_oi_amounts$view = factor(geneset_oi_amounts$view, levels = sort(unique(geneset_oi_amounts$view)))# recode to ensure right ordering

            absolute_plot_1_perc[[i]] = ggplot(data = geneset_oi_amounts[(geneset_oi_amounts$variable == i),], aes(x = view, y = n, fill = view)) +
            xlabel + 
            ylabel + 
            plot_config + 
            geom_bar(stat="identity") + coord_flip()  + theme(legend.position = 'none')+ 
            ggtitle(paste0('Top ', 2*as.numeric( top_var_fraction) *100, '% of features')) + scale_fill_manual(values = type_color_codes$color_code) # TBD: maybe improve even with default value + specifying all colors via table
        }
    
    barplot_top_features_absolute[[j]] = absolute_plot_1_perc
       

}  

In [828]:
#barplot_top_features_percentages[[1]]

In [829]:
## Combine the plots and save

In [830]:
# Specify the figure name
figure_name = paste0( "FIG04_Top_Feature_Overview_per_Factor")

In [831]:
# Specify the sizes of the plot
width_par = 8.07
height_par =2.8

In [832]:
for(j in 1:nrow(factor_configs)){
    mofa_name = factor_configs$mofa_result_name[j]
    
    # get the relvant plot
    explained_variance_heatmap = variance_plots[[j]]
    absolute_plot_1_perc = barplot_top_features_absolute[[j]]
    percentage_plot_1_perc = barplot_top_features_percentages[[j]]


    pdf(paste0('figures/04_figures/', figure_name, '_',   mofa_name, '.pdf'), width =width_par, height =height_par)
    for( i in 1:length(explained_variance_heatmap)){
        legend = get_legend(explained_variance_heatmap[[i]])

        combined1 = ggarrange(explained_variance_heatmap[[i]] + theme(legend.position = 'none'),
                             absolute_plot_1_perc[[i]] + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank() ), 
                             percentage_plot_1_perc[[i]] + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank() ),  
                             nrow=1, widths = c(2.2,1,1))
        combined1 = annotate_figure(combined1, right = legend)

        print( combined1)
        }
    dev.off()   
    }